In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

"""
import pandas as pd
#from IPython.core.display import display, HTML
#display(HTML('<h1>Hello, world!</h1>'))
"""

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1557093610127_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
"\nimport pandas as pd\n#from IPython.core.display import display, HTML\n#display(HTML('<h1>Hello, world!</h1>'))\n"

In [2]:
spark.sql("show databases").show(truncate=False)
spark.sql("show tables in secondary").show(truncate=False)

+--------------+
|databaseName  |
+--------------+
|cust_complains|
|default       |
|secondary     |
+--------------+

+---------+----------+-----------+
|database |tableName |isTemporary|
+---------+----------+-----------+
|secondary|census_all|false      |
|secondary|census_raw|false      |
+---------+----------+-----------+

In [3]:
spark.sql("select * from secondary.census_all").show(2)

+----------+-----------+-----------+------+-------+--------------+
|population|minimum_age|maximum_age|gender|zipcode|        geo_id|
+----------+-----------+-----------+------+-------+--------------+
|        50|         30|         34|female|  61747|8600000US61747|
|         5|         85|       null|  male|  64120|8600000US64120|
+----------+-----------+-----------+------+-------+--------------+
only showing top 2 rows

In [4]:
spark.sql("select geo_id,population,minimum_age,maximum_age,gender,zipcode from secondary.census_all WHERE zipcode = '61748'").show()

+--------------+----------+-----------+-----------+------+-------+
|        geo_id|population|minimum_age|maximum_age|gender|zipcode|
+--------------+----------+-----------+-----------+------+-------+
|8600000US61748|        70|         25|         29|  male|  61748|
|8600000US61748|        49|         62|         64|female|  61748|
|8600000US61748|       100|         35|         39|  male|  61748|
|8600000US61748|        22|         80|         84|female|  61748|
|8600000US61748|       117|         45|         49|  male|  61748|
|8600000US61748|        36|         65|         66|female|  61748|
|8600000US61748|        16|         80|         84|  male|  61748|
|8600000US61748|        42|         67|         69|female|  61748|
|8600000US61748|        95|          5|          9|female|  61748|
|8600000US61748|       104|         55|         59|  male|  61748|
|8600000US61748|        36|         67|         69|  male|  61748|
|8600000US61748|         8|         21|         21|female|  61

In [34]:
spark.sql("select geo_id,population,minimum_age,maximum_age,gender,zipcode from secondary.census_all WHERE minimum_age is null AND maximum_age is null AND gender = '' ORDER BY zipcode").show()

+--------------+----------+-----------+-----------+------+-------+
|        geo_id|population|minimum_age|maximum_age|gender|zipcode|
+--------------+----------+-----------+-----------+------+-------+
|8600000US00602|     41520|       null|       null|      |    602|
|8600000US00603|     54689|       null|       null|      |    603|
|8600000US00606|      6615|       null|       null|      |    606|
|8600000US00610|     29016|       null|       null|      |    610|
|8600000US00612|     67010|       null|       null|      |    612|
|8600000US00616|     11017|       null|       null|      |    616|
|8600000US00617|     24597|       null|       null|      |    617|
|8600000US00622|      7853|       null|       null|      |    622|
|8600000US00623|     43061|       null|       null|      |    623|
|8600000US00624|     26241|       null|       null|      |    624|
|8600000US00627|     35159|       null|       null|      |    627|
|8600000US00631|      1852|       null|       null|      |    

In [35]:
spark.sql("select count(zipcode) from secondary.census_all WHERE minimum_age is null AND maximum_age is null AND gender= ''").show()

+--------------+
|count(zipcode)|
+--------------+
|         33119|
+--------------+

In [7]:
spark.sql('select count(distinct zipcode) from secondary.census_all').show()

+-----------------------+
|count(DISTINCT zipcode)|
+-----------------------+
|                  33119|
+-----------------------+

In [9]:
populationDF = spark.sql("select population, zipcode as c_zipcode from secondary.census_all WHERE minimum_age is null AND maximum_age is null AND gender= ''")

In [10]:
populationDF.columns

['population', 'c_zipcode']

In [11]:
populationDF.head()

Row(population=27669, c_zipcode=91324)

In [12]:
customerDF = spark.sql("SELECT * FROM cust_complains.cust_complaints_all")

In [13]:
customerDF.columns

['date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company_public_response', 'company', 'state', 'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response', 'timely_response', 'consumer_disputed', 'complaint_id']

In [14]:
joinExpression = populationDF["c_zipcode"] == customerDF['zip_code']

In [15]:
joinType = "right_outer"
customer_populationDF = customerDF.join(populationDF, joinExpression, joinType)

In [19]:
customer_populationDF.show(5)

+-------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+--------------+-------------------------+-------------+--------------------+--------------------+---------------+-----------------+------------+----------+---------+
|date_received|             product|         sub_product|               issue|           sub_issue|consumer_complaint_narrative|company_public_response|             company|state|zip_code|          tags|consumer_consent_provided|submitted_via|date_sent_to_company|    company_response|timely_response|consumer_disputed|complaint_id|population|c_zipcode|
+-------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+--------------+-------------------------+-------------+--------------------+--------------------+-------

In [16]:
income = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").load("s3://snehalrawinput/income2016.csv")

In [17]:
incomeDF = income['ZIPCODE','A02650','NUMDEP']

In [29]:
#val dfRenamed = df.toDF(newNames: _*)

incomeDF_renamed = incomeDF.withColumnRenamed('ZIPCODE','zipcode_income')
incomeDF_renamed = incomeDF_renamed.withColumnRenamed('A02650','Total_Income')
#incomeDF_renamed = incomeDF_renamed.withColumnRenamed('NUMDEP',)

In [30]:
incomeDF_renamed.columns

['zipcode_income', 'Total_Income', 'NUMDEP']

In [62]:
#incomeDF.createOrReplaceTempView("income")

In [63]:
#spark.sql("show tables in cust_complains").show(truncate=False)
#spark.sql("show tables in secondary").show(truncate=False)

In [31]:
joinExpression2 = incomeDF_renamed["zipcode_income"] == customer_populationDF['zip_code']

In [32]:
joinType = "right_outer"
cc_pop_inc = customer_populationDF.join(incomeDF_renamed, joinExpression2, joinType)

In [33]:
cc_pop_inc.columns

['date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company_public_response', 'company', 'state', 'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response', 'timely_response', 'consumer_disputed', 'complaint_id', 'population', 'c_zipcode', 'zipcode_income', 'Total_Income', 'NUMDEP']

In [36]:
cc_pop_inc.write.parquet("s3://snehalpreprocessedfiles/Dataframes_equi_wells/cc_pop_inc_5.5.pq")

In [37]:
#cc_april = cc_pop_inc['date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'company_public_response', 'company', 'state', 'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response', 'timely_response', 'consumer_disputed', 'complaint_id', 'A02650', 'NUMDEP','avg_population']

In [38]:
#cc_april.printSchema()

In [69]:
#cc_april.write.parquet("s3://snehalpreprocessedfiles/cc_april.pq")

In [40]:
cc_pop_inc_agg = cc_pop_inc.groupBy("zip_code").agg(count("zip_code").alias("Number_of_Complaints"))

In [44]:
cc_pop_inc_agg = cc_pop_inc_agg.withColumnRenamed('zip_code','zipcode_agg')

In [45]:
cc_pop_inc_agg.columns

['zipcode_agg', 'Number_of_Complaints']

In [46]:
joinExpression3 = cc_pop_inc_agg["zipcode_agg"] == cc_pop_inc['zip_code']

In [47]:
joinType = "inner"
cc_pop_inc = cc_pop_inc.join(cc_pop_inc_agg, joinExpression3, joinType)

In [48]:
cc_pop_inc.columns

['date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company_public_response', 'company', 'state', 'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response', 'timely_response', 'consumer_disputed', 'complaint_id', 'population', 'c_zipcode', 'zipcode_income', 'Total_Income', 'NUMDEP', 'zipcode_agg', 'Number_of_Complaints']

In [49]:
cc_pop_inc.select('zip_code', 'Number_of_Complaints', 'population', 'Total_Income').show(5)

+--------+--------------------+----------+------------+
|zip_code|Number_of_Complaints|population|Total_Income|
+--------+--------------------+----------+------------+
|   11563|                 540|     22666|      286397|
|   11563|                 540|     22666|      380004|
|   11563|                 540|     22666|      118807|
|   11563|                 540|     22666|      106785|
|   11563|                 540|     22666|       75937|
+--------+--------------------+----------+------------+
only showing top 5 rows

In [51]:
cc_pop_inc.createOrReplaceTempView("cc_pop_inc")


In [64]:
avg_inc = spark.sql("select zip_code, Number_of_Complaints, population, Ceil(AVG(Total_Income)) average_income FROM cc_pop_inc GROUP BY zip_code, Number_of_Complaints, population")

In [91]:
avg_inc.columns

['zip_code', 'Number_of_Complaints', 'population', 'average_income']

In [92]:
avg_inc.columns

['zip_code', 'Number_of_Complaints', 'population', 'average_income']

In [97]:
avg_inc.createOrReplaceTempView("avg_inc")

In [96]:
avg_inc.select('zip_code', 'Number_of_Complaints', 'population').show(5)

+--------+--------------------+----------+
|zip_code|Number_of_Complaints|population|
+--------+--------------------+----------+
|   11563|                 540|     22666|
|   11722|                 756|     35177|
|   12529|                   6|      2859|
|   14204|                  54|      8691|
|   14838|                  18|      1956|
+--------+--------------------+----------+
only showing top 5 rows

Scaling factor of 10K
10000/30,067

In [101]:
spark.sql("SELECT zip_code, Number_of_Complaints/population*10000 as Average_Complaints FROM avg_inc ORDER BY Number_of_Complaints/population*10000 ").show(50)

+--------+------------------+
|zip_code|Average_Complaints|
+--------+------------------+
|   75964| 2.985520226899537|
|   54501| 2.997152704930316|
|   49091| 3.019323671497584|
|   13901|3.0344409042633895|
|   44654|3.0489354133848265|
|   85546| 3.049245311785333|
|   49783|3.0506406345332517|
|   77011| 3.069524735253492|
|   47710|3.0726686126901215|
|   49735|3.0745580322828596|
|   89706|3.0753459764223474|
|   79045| 3.081189339084887|
|   46012|3.0905532090244154|
|   53094| 3.130053732589076|
|   82007| 3.142019271051529|
|   17745| 3.147458427319939|
|   80751|  3.18184228668399|
|   29307|  3.19982934243507|
|   31750|3.2107882485150103|
|   72801|3.2111319240032112|
|   73036|3.2266738370529713|
|   44622|3.2278889606197545|
|   94519| 3.251503820516989|
|   38901| 3.260337988371461|
|   57301|3.2653061224489797|
|   29627|3.2667283715359066|
|   40361|3.2930845225027445|
|   16201|3.2941693203030638|
|   67208|3.3037828313418864|
|   44820|3.3145508783559827|
|   35903|

In [105]:
spark.sql("SELECT zip_code, Number_of_Complaints/population*10000 as Average_Complaints, average_income FROM avg_inc").show(2)

+--------+------------------+--------------+
|zip_code|Average_Complaints|average_income|
+--------+------------------+--------------+
|   11563|238.24230124415425|        167009|
|   11722|214.91315348096768|        136987|
+--------+------------------+--------------+
only showing top 2 rows

Adding a scale of 10000

In [108]:
avgcc = spark.sql("SELECT zip_code as zipcode_avgcc, CEIL(Number_of_Complaints/population*10000) as Average_Complaints, Average_Income FROM avg_inc")

In [111]:
avgcc.columns

['zipcode_avgcc', 'Average_Complaints', 'Average_Income']

In [112]:
joinExpression4 = avgcc["zipcode_avgcc"] == cc_pop_inc['zip_code']
joinType = "inner"
avgcc_inc_All_5May = cc_pop_inc.join(avgcc, joinExpression4, joinType)
avgcc_inc_All_5May.columns

['date_received', 'product', 'sub_product', 'issue', 'sub_issue', 'consumer_complaint_narrative', 'company_public_response', 'company', 'state', 'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via', 'date_sent_to_company', 'company_response', 'timely_response', 'consumer_disputed', 'complaint_id', 'population', 'c_zipcode', 'zipcode_income', 'Total_Income', 'NUMDEP', 'zipcode_agg', 'Number_of_Complaints', 'zipcode_avgcc', 'Average_Complaints', 'Average_Income']

In [113]:
avgcc_inc_All_5May.write.parquet("s3://snehalpreprocessedfiles/Dataframes_equi_wells/avgcc_inc_All_5May.pq")